<a href="https://colab.research.google.com/github/Aashiff2003/DSGP-group-25/blob/Geethmi/Yolo_with_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/107.7 GB disk)


In [ ]:
import torch
import torchvision.models as models
from ultralytics import YOLO
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DSGP/Model

/content/drive/MyDrive/DSGP/Model


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Load YOLOv8 model
yolo_model = YOLO("yolov8s.yaml")

# Load Pretrained ResNet Model as Feature Extractor
resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Remove the final classification layer of ResNet (we only need feature maps)
resnet_backbone = torch.nn.Sequential(*list(resnet.children())[:-2])

# Replace YOLO’s backbone with ResNet
yolo_model.model.backbone = resnet_backbone

# Save the modified model
yolo_model.save("yolov8_resnet.pt")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 93.9MB/s]


In [ ]:
!ls -lh /content/drive/MyDrive/DSGP/Model

total 94M
-rw------- 1 root root  60K Jan 29 06:30 Bird_Detection.ipynb
drwx------ 2 root root 4.0K Jan 29 22:47 data
drwx------ 2 root root 4.0K Jan 29 22:47 data_old
drwx------ 2 root root 4.0K Jan 24 11:03 dataset
-rw------- 1 root root  142 Jan 29 22:04 data.yaml
drwx------ 2 root root 4.0K Jan 24 14:05 runs
-rw------- 1 root root 5.4M Jan 30 11:55 yolo11n.pt
-rw------- 1 root root  67M Jan 30 13:44 yolov8_resnet.pt
-rw------- 1 root root  22M Jan 24 14:05 yolov8s.pt
-rw------- 1 root root 9.3K Jan 30 13:20 Yolo_with_ResNet50.ipynb


In [ ]:
!yolo task=detect mode=train model=yolov8_resnet.pt data=data.yaml epochs=50 imgsz=640 plots=True batch=16 lr0=0.001

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8_resnet.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

In [ ]:
# !yolo task=detect mode=val model=runs/detect/train9/weights/best.pt data=data.yaml

In [ ]:
# !yolo task=detect mode=predict model=runs/detect/train9/weights/best.pt conf=0.25 source=data/test/images

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load image
image_path = "/content/drive/MyDrive/DSGP/Model/data/train/images/20201111_sljc_1_0000043.png"
label_path = "/content/drive/MyDrive/DSGP/Model/data/train/labels/20201111_sljc_1_0000043.txt"

# Read the image
image = cv2.imread(image_path)
height, width, _ = image.shape

# Read YOLO annotation file
with open(label_path, "r") as file:
    lines = file.readlines()

# Draw bounding boxes
for line in lines:
    values = line.strip().split()
    x_center, y_center, bbox_width, bbox_height = map(float, values[1:])

    # Reduce bounding box size by a factor (adjust factor as needed)
    shrink_factor = 0.5  # Reduce the box size to 50%
    bbox_width *= shrink_factor
    bbox_height *= shrink_factor

    # Convert YOLO format (normalized) to pixel values
    x_center, y_center, bbox_width, bbox_height = (
        x_center * width,
        y_center * height,
        bbox_width * width,
        bbox_height * height,
    )

    # Convert to top-left corner format
    x1 = int(x_center - bbox_width / 2)
    y1 = int(y_center - bbox_height / 2)
    x2 = int(x_center + bbox_width / 2)
    y2 = int(y_center + bbox_height / 2)

    # Draw the bounding box (thin and smaller)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)  # Reduce thickness to 1

# Display image with bounding boxes
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

